In [1]:
#Set parameters
params = {}

params['alpha'] = -1.3
params['sigma_M'] = 0.2
params['mpeak_cut'] = 8.
params['B'] = 1.

In [3]:
# %load load_hyperparams.py
import pickle

def load_hyperparams():
    """
    Returns hyperparameters, cosmological parameters, and loads data for predict_satellites.py and predict_orphans.py
    """
    #Load halo data (encoding='latin1' for Python3)
    with open('halo_data.pkl', 'rb') as halo_input:
        halo_data = pickle.load(halo_input, encoding='latin1')

    #Load interpolator
    with open('interpolator.pkl', 'rb') as interp:
        vpeak_Mr_interp = pickle.load(interp, encoding='latin1')

    #Cosmological params
    cosmo_params = {}
    cosmo_params['omega_b'] = 0.0 
    cosmo_params['omega_m'] = 0.286
    cosmo_params['h'] = 0.7

    #hyperparameters
    hparams = {}
    hparams['vpeak_cut'] = 10.
    hparams['vmax_cut'] = 9.
    hparams['sigma_M_min'] = 10**-5
    hparams['chi'] = 1.
    hparams['A'] = 0.02
    hparams['gamma'] = -0.7
    hparams['beta'] = 0.
    hparams['sigma_r'] = 0.01
    hparams['size_min'] = 0.02
    hparams['O'] = 1.
    ###
    return hparams, cosmo_params, halo_data, vpeak_Mr_interp 


In [4]:
hparams, cosmo_params, halo_data, vpeak_Mr_interp = load_hyperparams()

In [6]:
# %load predict_satellites.py
import numpy as np

def satellite_properties(halo_data,params,hparams,cosmo_params,vpeak_Mr_interp):
    """
    Returns properties of luminous satellites corresponding to surviving subhalos at z=0 in a DMO zoom-in simulation

    Args:
	halo_data (dict): dict containing properties of host and subhalos
 
        params (dict): dict containing free parameters
        params['alpha'] (float): faint-end slope of satellite luminosity function
	params['sigma_M'] (float): lognormal scatter in M_V--V_peak relation (in dex)
	params['mpeak_cut'] (float): mass threshold for galaxy formation (in log10(M*/h))
	params['B'] (float): subhalo disruption probability (due to baryons)
	
        hparams (dict): dict containing hyperparameters
        hparams['vpeak_cut']: subhalo vpeak resolution threshold
        hparams['vmax_cut']: subhalo vmax resolution threshold
        hparams['sigma_M_min']: minimum allowed luminosity scatter
        hparams['chi']: satellite radial scaling
        hparams['A']: satellite size relation normalization
        hparams['gamma']: satellite size relation slope
        hparams['beta']: tidal stripping parameter
        hparams['sigma_r']: scatter in satellite size relation
        hparams['size_min']: minimum allowed satellite size
        hparams['O']: orphan satellite parameter

        cosmo_params (dict): dict containing cosmological parameters
        cosmo_params['omega_b']: baryon fraction
        cosmo_params['omega_m']: matter fraction
        cosmo_params['h']: dimensionless hubble parameter

        vpeak_Mr_interp (interpolating function): implements Mv--Vpeak abund        ance matching relation

    Returns:
	satellite_properties (dict): dict containing r-band absolute magnitu        des, Galactocentric radii, half-light radii, disruption probabilitie        s, and 3D Galactocentric positions of luminous satellites
    """
    #Define output dict
    satellite_properties = {}
    #Cut subhalo catalogs
    Halo_subs = halo_data['Halo_subs']
    mpeak_idx = (Halo_subs['mpeak']*(1.-cosmo_params['omega_b']/cosmo_params['omega_m'])>10**(params['mpeak_cut']))
    vpeak_idx = (Halo_subs['vpeak']>hparams['vpeak_cut'])
    vmax_idx = (Halo_subs['vmax']>hparams['vmax_cut'])
    cut_idx = mpeak_idx & vpeak_idx & vmax_idx
    Halo_subs_cut = Halo_subs[cut_idx]
    #Calculate luminosities
    sort_idx = np.argsort(np.argsort(Halo_subs_cut['vpeak']))
    satellite_properties['Mr'] = (np.log(np.random.lognormal(vpeak_Mr_interp(Halo_subs_cut['vpeak'],params['alpha']),(np.log(10)*params['sigma_M']).clip(min=hparams['sigma_M_min']))))[sort_idx]
    #Calculate positions
    Halo_main = halo_data['Halo_main']
    Halox = hparams['chi']*(Halo_subs_cut['x']-Halo_main[0]['x'])*(1000/cosmo_params['h'])
    Haloy = hparams['chi']*(Halo_subs_cut['y']-Halo_main[0]['y'])*(1000/cosmo_params['h'])
    Haloz = hparams['chi']*(Halo_subs_cut['z']-Halo_main[0]['z'])*(1000/cosmo_params['h'])
    satellite_properties['radii'] = np.sqrt(Halox**2 + Haloy**2 + Haloz**2)
    satellite_properties['pos'] = np.vstack((Halox,Haloy,Haloz)).T
    #Calculate sizes
    c = halo_data['rvir']/halo_data['rs']
    Halo_r12 = (hparams['A']*(c[cut_idx]/10.0)**(hparams['gamma'])*halo_data['rvir'][cut_idx]/cosmo_params['h'])**(((Halo_subs_cut['vmax']/Halo_subs_cut['vacc']).clip(max=1.0))**hparams['beta'])
    satellite_properties['r12'] = np.log(np.random.lognormal(Halo_r12,np.log(10)*hparams['sigma_r'])).clip(min=hparams['size_min']) 
    #Calculate disruption probabilities
    satellite_properties['prob'] = halo_data['Halo_ML_prob'][cut_idx]**(1./params['B'])
    ###
    return satellite_properties

In [7]:
#Return satellite properties for a particular host halo
satellite_properties(halo_data[0], params, hparams, cosmo_params, vpeak_Mr_interp)

{'Mr': array([ -1.47291203e+01,  -1.32091507e+01,  -1.46014114e+01,
         -1.00403720e+01,  -1.03774367e+01,  -7.77081694e+00,
         -6.98591494e+00,  -7.39158203e+00,  -7.99147394e+00,
         -9.76702197e+00,  -1.30581657e+01,  -1.43043544e+01,
         -6.28652692e+00,  -7.13391424e+00,  -8.53794430e+00,
         -9.65940605e+00,  -1.09593109e+01,  -9.60779349e+00,
         -3.65621491e+00,  -5.92707263e+00,  -1.06410566e+00,
         -8.09479013e+00,  -8.79300093e+00,  -5.30646654e+00,
         -4.05908359e+00,  -6.93690885e+00,  -1.28172447e+01,
         -9.00599235e+00,  -6.94792344e+00,  -2.52432423e+00,
         -1.03997135e+01,  -9.65988376e+00,  -6.56100367e+00,
         -3.01318093e+00,  -1.93078599e+00,  -1.84290100e+00,
         -4.77491018e+00,  -1.61205890e+00,  -2.76315523e+00,
          2.08990041e+00,  -1.92810467e+00,  -2.85999761e+00,
         -1.25207707e+00,  -6.56419141e+00,  -6.38971988e-01,
         -6.98049020e+00,   2.40892807e+00,   1.33791068e+00,
  

In [9]:
# %load predict_orphans.py
import numpy as np

def orphan_satellite_properties(halo_data,params,hparams,cosmo_params,vpeak_Mr_interp):
    """
    Returns properties of luminous orphan satellites corresponding to disrupted subhalos that have been tracked to z=0 in a DMO zoom-in simulation

    Args:
	halo_data (dict): dict containing properties of host and subhalos 

        params (dict): dict containing free parameters
        params['alpha'] (float): faint-end slope of satellite luminosity function
	params['sigma_M'] (float): lognormal scatter in M_V--V_peak relation (in dex)
	params['mpeak_cut'] (float): mass threshold for galaxy formation (in log10(M*/h))
	
        hparams (dict): dict containing hyperparameters
        hparams['vpeak_cut']: subhalo vpeak resolution threshold
        hparams['vmax_cut']: subhalo vmax resolution threshold
        hparams['sigma_M_min']: minimum allowed luminosity scatter
        hparams['chi']: satellite radial scaling
        hparams['A']: satellite size relation normalization
        hparams['gamma']: satellite size relation slope
        hparams['beta']: tidal stripping parameter
        hparams['sigma_r']: scatter in satellite size relation
        hparams['size_min']: minimum allowed satellite size
        hparams['O']: orphan satellite parameter

        cosmo_params (dict): dict containing cosmological parameters
        cosmo_params['omega_b']: baryon fraction
        cosmo_params['omega_m']: matter fraction
        cosmo_params['h']: dimensionless hubble parameter

        vpeak_Mr_interp (interpolating function): implements Mv--Vpeak abund        ance matching relation

    Returns:
	orphan_satellite_properties (dict): dict containing r-band absolute         magnitudes,	Galactocentric radii, half-light radii, disruption proba        bilities, and 3D Galactocentric positions of luminous satellites
    """
    #Define output dict
    orphan_satellite_properties = {}
    #Cut subhalo catalogs
    Halo_subs = halo_data['orphan_catalog']
    mpeak_idx = (halo_data['orphan_catalog_mpeak']*(1.-cosmo_params['omega_b']/cosmo_params['omega_m'])>10**(params['mpeak_cut']))
    radii_idx = Halo_subs[:,3] < 300
    cut_idx = mpeak_idx & radii_idx
    Halo_subs_cut = Halo_subs[cut_idx]
    #Calculate luminosities
    sort_idx = np.argsort(np.argsort(Halo_subs_cut[:,5]))
    orphan_satellite_properties['Mr'] = (np.log(np.random.lognormal(vpeak_Mr_interp(Halo_subs_cut[:,5],params['alpha']),(np.log(10)*params['sigma_M']).clip(min=hparams['sigma_M_min']))))[sort_idx]
    #Calculate positions
    Halo_main = halo_data['Halo_main']
    Halox = hparams['chi']*(Halo_subs_cut[:,0]-(Halo_main[0]['x']*1000/cosmo_params['h']))
    Haloy = hparams['chi']*(Halo_subs_cut[:,1]-(Halo_main[0]['y']*1000/cosmo_params['h']))
    Haloz = hparams['chi']*(Halo_subs_cut[:,2]-(Halo_main[0]['z']*1000/cosmo_params['h']))
    orphan_satellite_properties['radii'] = Halo_subs_cut[:,3]
    orphan_satellite_properties['pos'] = np.vstack((Halox,Haloy,Haloz)).T
    #Calculate sizes
    c = halo_data['rvir_orphan']/halo_data['rs_orphan']
    Halo_r12 = (hparams['A']*(c[cut_idx]/10.0)**(hparams['gamma'])*halo_data['rvir_orphan'][cut_idx]/cosmo_params['h'])**(((Halo_subs_cut[:,4]/Halo_subs_cut[:,9]).clip(max=1.0))**hparams['beta'])
    orphan_satellite_properties['r12'] = np.log(np.random.lognormal(Halo_r12,np.log(10)*hparams['sigma_r'])).clip(min=hparams['size_min']) 
    #Calculate disruption probabilities
    orphan_satellite_properties['prob'] = (1.-halo_data['orphan_aacc'][cut_idx])**(hparams['O'])
    ###
    return orphan_satellite_properties

In [10]:
#Return orphan satellite properties for a particular host halo
orphan_satellite_properties(halo_data[0], params, hparams, cosmo_params, vpeak_Mr_interp)

{'Mr': array([ -0.56002771,   1.36414403,  -0.83323949,  -1.41299712,
         -1.69028532,  -7.66647375,  -0.33027937,   2.41937819,
          2.06964953,  -4.82950056,  -9.81391713,  -4.55433594,
        -13.80825734,   2.60770817,  -1.89919779,   4.36616807,
          1.10923556,  -0.38385465,  -2.03367302,  -8.85449388,
        -12.94378758, -12.69367704,  -0.56631741,  -0.15248869,
          1.90330783,  -1.76980291,  -1.50022146,  -3.05134807,
         -8.656927  ,   1.19456596, -12.46376382,  -2.0371622 ,
          0.09498716,   0.80375908,  -0.22995637,  -1.39888053,
         -7.87897775,   0.40029659,  -4.95424106,  -0.30900432,  -0.87818452]),
 'pos': array([[  2.32326966e+01,   3.05892137e+01,  -1.10938862e+01],
        [ -2.92998417e+01,   9.86465985e+01,  -8.07012776e+00],
        [ -6.90219566e+01,  -6.94996041e+01,   3.37397512e+01],
        [  4.09791702e+01,   1.46246077e+01,   1.88804119e+01],
        [  7.91409545e+01,  -3.81932523e+01,   4.95403433e+00],
        [  